# Trabalho 2

## 4.1.7 c)

Neste notebook está o código relativo à MLPRegressor.

### Imports

In [ ]:
from IPython.display import clear_output
import operator
import warnings
from time import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import utilidades as ut

### Inicializações e variáveis

In [ ]:
warnings.filterwarnings("ignore")  # Desabilitar warnings.
plt.style.use(
    "style/estilo.mplstyle")  # Garantir que se utiliza um estilo definido centralmente e comum a todos os gráficos.
%matplotlib inline

label_encoder = LabelEncoder()

ficheiro = "dados_preparados.csv"
colunas_numericas = ["Idade", "FCV", "NRP", "CA", "FAF", "TUDE", "IMC"]
colunas_classes = ["Genero", "Historico_obesidade_familiar", "FCCAC", "Fumador", "MCC", "CCER", "CBA", "TRANS"]

## Leitura dos dados preparados

In [ ]:
dados_trabalho = pd.read_csv(ficheiro)

Remover o atributo *Label*, que é uma classificação de obesidade.

In [ ]:
dados_trabalho.drop(["Label"], axis=1, inplace=True)

In [ ]:
dados_trabalho

Temos no entanto de realizar primeiro o encoding das classes para valores numéricos, esta operação é realizada usando o ``sklearn.preprocessing.LabelEncoder`` em todas as colunas com apenas 2 classes.

In [ ]:
ut.titulo("Valores codificados por atributo")

for coluna in colunas_classes:
    if dados_trabalho[coluna].dtype == 'object':
        dados_trabalho[coluna] = label_encoder.fit_transform(dados_trabalho[coluna].values)
        ut.etiqueta_e_valor(coluna, str(sorted(dados_trabalho[coluna].unique())))

## Normalização



In [ ]:
dados_normalizados = MinMaxScaler().fit_transform(dados_trabalho)
dataframe_normalizado = pd.DataFrame(
    dados_normalizados,
    columns=dados_trabalho.columns
)
dataframe_normalizado

In [ ]:
dataframe_normalizado

| Target | Preditores                   |
|--------|------------------------------|
| IMC    | Todos os restantes atributos |

Ver TP5

O target não muda

In [ ]:
y = dataframe_normalizado.IMC

## Preditores

In [ ]:
lista_preditores = list(dataframe_normalizado.columns)
lista_preditores.remove("IMC")
X = dataframe_normalizado[lista_preditores]

### Holdout

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=100)

## Cálculo

In [ ]:
ativacao = ["identity", "logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]
n_hidden = [
    2, 3, 4, 5, 6, 7, 8, 9, 
    [4, 2], [5, 2], [6, 2], 
    [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], 
    [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], 
    [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], 
    [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], 
    [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6],
    [2, 2, 2], [2, 2, 3], [2, 2, 4], [2, 2, 5], [2, 2, 6],
    [2, 3, 2], [2, 3, 3], [2, 3, 4], [2, 3, 5], [2, 3, 6],
    [2, 4, 2], [2, 4, 3], [2, 4, 4], [2, 4, 5], [2, 4, 6],
    [2, 5, 2], [2, 5, 3], [2, 5, 4], [2, 5, 5], [2, 5, 6],
    [2, 6, 2], [2, 6, 3], [2, 6, 4], [2, 6, 5], [2, 6, 6],
    [3, 2, 2], [3, 2, 3], [3, 2, 4], [3, 2, 5], [3, 2, 6],
    [3, 3, 2], [3, 3, 3], [3, 3, 4], [3, 3, 5], [3, 3, 6],
    [3, 4, 2], [3, 4, 3], [3, 4, 4], [3, 4, 5], [3, 4, 6],
    [3, 5, 2], [3, 5, 3], [3, 5, 4], [3, 5, 5], [3, 5, 6],
    [3, 6, 2], [3, 6, 3], [3, 6, 4], [3, 6, 5], [3, 6, 6],
    [4, 2, 2], [4, 2, 3], [4, 2, 4], [4, 2, 5], [4, 2, 6],
    [4, 3, 2], [4, 3, 3], [4, 3, 4], [4, 3, 5], [4, 3, 6],
    [4, 4, 2], [4, 4, 3], [4, 4, 4], [4, 4, 5], [4, 4, 6],
    [4, 5, 2], [4, 5, 3], [4, 5, 4], [4, 5, 5], [4, 5, 6],
    [4, 6, 2], [4, 6, 3], [4, 6, 4], [4, 6, 5], [4, 6, 6],
    [5, 2, 2], [5, 2, 3], [5, 2, 4], [5, 2, 5], [5, 2, 6],
    [5, 3, 2], [5, 3, 3], [5, 3, 4], [5, 3, 5], [5, 3, 6],
    [5, 4, 2], [5, 4, 3], [5, 4, 4], [5, 4, 5], [5, 4, 6],
    [5, 5, 2], [5, 5, 3], [5, 5, 4], [5, 5, 5], [5, 5, 6],
    [5, 6, 2], [5, 6, 3], [5, 6, 4], [5, 6, 5], [5, 6, 6],
    [6, 2, 2], [6, 2, 3], [6, 2, 4], [6, 2, 5], [6, 2, 6],
    [6, 3, 2], [6, 3, 3], [6, 3, 4], [6, 3, 5], [6, 3, 6],
    [6, 4, 2], [6, 4, 3], [6, 4, 4], [6, 4, 5], [6, 4, 6],
    [6, 5, 2], [6, 5, 3], [6, 5, 4], [6, 5, 5], [6, 5, 6],
    [6, 6, 2], [6, 6, 3], [6, 6, 4], [6, 6, 5], [6, 6, 6],
]

In [ ]:
def rede_neural(camadas, ativador, solv):
    modelo_nn = MLPRegressor(
        hidden_layer_sizes=camadas,
        activation=ativador,
        solver=solv,
        max_iter=1000,
        learning_rate_init=0.001
    )
    # Treino
    modelo_nn.fit(X_train, y_train)

    # Métricas nos dados de Teste
    y_pred_teste = modelo_nn.predict(X_test)
    rsquared_teste = modelo_nn.score(X_test, y_test)
    rmse_teste = np.sqrt(mean_squared_error(y_test, y_pred_teste))

    # Métricas nos dados de Treino.
    y_pred_treino = modelo_nn.predict(X_train)  
    rsquared_treino = modelo_nn.score(X_train, y_train)
    rmse_treino = np.sqrt(mean_squared_error(y_train, y_pred_treino))
    return ativador, solv, camadas, rsquared_teste, rmse_teste, rsquared_treino, rmse_treino

## Cálculos

In [ ]:
resultados = pd.DataFrame(columns=["ativador", "solv", "camadas", "rsquared_teste", "rmse_teste", "rsquared_treino", "rmse_treino"])
inicio = time()
total = len(n_hidden) * len(solvers) * len(ativacao)
ciclo = 0

for ativador in ativacao:
    for solver in solvers:
        for niveis in n_hidden:
            resultados.loc[ciclo] = rede_neural(niveis, ativador, solver)
            ciclo += 1
            ut.texto_mesma_linha(f"Ciclo {ciclo}/{total} || Activador={ativacao} | Solver={solver} >> {(time() - inicio):.4f}s")

## Valores do melhor RMSE

Obter os 10 melhores

In [ ]:
best_n_rmse = resultados.nsmallest(10, 'rmse_teste')
best_n_rmse

In [ ]:
parametros_minimo = resultados.iloc[resultados["rmse_teste"].idxmin()]

In [ ]:
ut.titulo("Parametrizaçao e métricas onde temos o menor RMSE de teste")
ut.etiqueta_e_valor("Ocorre com: camadas", f"ativador = {parametros_minimo['ativador']} | solver = {parametros_minimo['solv']} | camadas = {parametros_minimo['camadas']}")
ut.etiqueta_e_valor("Nos dados de teste", f"Rsquared = {parametros_minimo['rsquared_treino']:.6f} RMSE = {parametros_minimo['rmse_treino']:.6f}")
ut.etiqueta_e_valor("Nos dados de treino", f"Rsquared = {parametros_minimo['rsquared_teste']:.6f} RMSE = {parametros_minimo['rmse_teste']:.6f}")

## Cálculo melhores parâmetros

Repetição dos melhores cálculos para apresentação dos detalhes da respetiva rede.

In [ ]:
modelo_nn = MLPRegressor(
    hidden_layer_sizes=parametros_minimo["camadas"],
    activation=parametros_minimo["ativador"],
    solver=parametros_minimo["solv"],
    max_iter=1000,
    learning_rate_init=0.001
)

modelo_nn.fit(X_train, y_train)  # Treino
y_pred = modelo_nn.predict(X_test)  # Teste


#### Pesos entre as várias camadas

In [ ]:
print(modelo_nn.coefs_)